In [0]:
%sql
CREATE OR REPLACE TABLE qualified_country_list (
  TeamId INTEGER,
  TeamName VARCHAR(255),
  TeamGroup VARCHAR(255)
);

INSERT INTO qualified_country_list (TeamId, TeamName, TeamGroup)
VALUES
(1, 'France', 'A'),
(2, 'Italy', 'A'),
(3, 'Belgium', 'A'),
(4, 'Portugal', 'A'),
(5, 'Spain', 'B'),
(6, 'Netherlands', 'B'),
(7, 'Germany', 'B'),
(8, 'England', 'B'),
(9, 'Turkey', 'C'),
(10, 'Austria', 'C'),
(11, 'Switzerland', 'C'),
(12, 'Denmark', 'C'),
(13, 'Croatia', 'D'),
(14, 'Ukraine', 'D'),
(15, 'Sweden', 'D'),
(16, 'Poland', 'D'),
(17, 'Wales', 'E'),
(18, 'Czech Republic', 'E'),
(19, 'Slovakia', 'E'),
(20, 'Hungary', 'E'),
(21, 'Finland', 'F'),
(22, 'North Macedonia', 'F'),
(23, 'Scotland', 'F'),
(24, 'Russia', 'F');

SELECT * FROM qualified_country_list;

TeamId,TeamName,TeamGroup
1,France,A
2,Italy,A
3,Belgium,A
4,Portugal,A
5,Spain,B
6,Netherlands,B
7,Germany,B
8,England,B
9,Turkey,C
10,Austria,C


In [0]:
%sql
create or replace table matches(
  MatchId int,
  TeamGroup varchar(20),
  home_country varchar(20),
  away_country varchar (20),
  home_country_goal int,
  away_country_goal int,
  winner_country varchar(20)
)

In [0]:
%sql
INSERT INTO matches
SELECT DISTINCT
row_number() OVER (ORDER BY 1) as MATCHID,
t1.TeamGroup as TeamGroup,
t1.TeamName AS home_country,
t2.TeamName As away_country,
(SELECT FLOOR(RAND() * 6) + 1) AS home_country_goal,
(SELECT FLOOR(RAND() * 6) + 1) AS away_country_goal,
'Initially Draw' AS winner_country
FROM
qualified_country_list as t1
JOIN
qualified_country_list as t2
ON t1.TeamGroup = t2.TeamGroup AND t1.TeamId < t2.TeamId;


select * from matches;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,4,4,Initially Draw
2,A,France,Belgium,1,1,Initially Draw
3,A,France,Italy,5,4,Initially Draw
4,A,Italy,Portugal,6,5,Initially Draw
5,A,Italy,Belgium,2,3,Initially Draw
6,A,Belgium,Portugal,1,5,Initially Draw
7,B,Spain,England,6,5,Initially Draw
8,B,Spain,Germany,3,5,Initially Draw
9,B,Spain,Netherlands,1,3,Initially Draw
10,B,Netherlands,England,3,5,Initially Draw


In [0]:
%sql
UPDATE matches SET winner_country = 
CASE
WHEN home_country_goal > away_country_goal THEN home_country
WHEN home_country_goal < away_country_goal THEN away_country
ELSE 'Draw'
END;

select * from group_stage;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,4,3,France
2,A,France,Belgium,1,4,Belgium
3,A,France,Italy,5,4,France
4,A,Italy,Portugal,4,2,Italy
5,A,Italy,Belgium,6,6,Draw
6,A,Belgium,Portugal,4,4,Draw
7,B,Spain,England,3,3,Draw
8,B,Spain,Germany,4,4,Draw
9,B,Spain,Netherlands,6,2,Spain
10,B,Netherlands,England,6,1,Netherlands


In [0]:
%sql
CREATE OR REPLACE TABLE group_stage_ranking AS
SELECT
  team_id as TeamName,
  TeamGroup,
  COUNT(*) AS Matches_Played,
  SUM (
    CASE
      WHEN goals_scored > goals_allowed THEN 1
      ELSE 0
    END
  ) AS Matches_won,
  SUM (
    CASE
      WHEN goals_scored = goals_allowed THEN 1
      ELSE 0
    END
  ) AS Matches_draw,
  SUM(
    CASE
      WHEN goals_scored < goals_allowed THEN 1
      ELSE 0
    END
  ) AS Matches_lost,
  SUM(goals_scored) AS Goals_scored,
  SUM(goals_allowed) AS Goals_allowed,
  SUM(goals_scored) - SUM(goals_allowed) AS Goal_difference,
  SUM(
    CASE
      WHEN goals_scored > goals_allowed THEN 3
      WHEN goals_scored = goals_allowed THEN 1
      ELSE 0
    END
  ) AS points
FROM(
    SELECT
      TeamGroup,
      home_country AS team_id,
      home_country_goal AS goals_scored,
      away_country_goal AS goals_allowed
    FROM
      group_stage
    UNION ALL
    SELECT
      TeamGroup,
      away_country AS team_id,
      away_country_goal AS goals_scored,
      home_country_goal AS goals_allowed
    FROM
      group_stage
  ) AS group_matches
GROUP BY
  team_id,
  TeamGroup

ORDER BY
  TeamGroup,
  points DESC;

select * FROM group_stage_ranking;

TeamName,TeamGroup,Matches_Played,Matches_won,Matches_draw,Matches_lost,Goals_scored,Goals_allowed,Goal_difference,points
France,A,3,2,0,1,10,11,-1,6
Belgium,A,3,1,2,0,14,11,3,5
Italy,A,3,1,1,1,14,13,1,4
Portugal,A,3,0,1,2,9,12,-3,1
Germany,B,3,1,2,0,11,9,2,5
Spain,B,3,1,2,0,13,9,4,5
Netherlands,B,3,1,0,2,9,10,-1,3
England,B,3,0,2,1,8,13,-5,2
Austria,C,3,3,0,0,10,6,4,9
Switzerland,C,3,1,0,2,9,10,-1,3


In [0]:
%sql
CREATE OR REPLACE TABLE round_of_16 AS
SELECT * FROM(
  SELECT TeamName, TeamGroup, points, Goal_difference,
  row_number() OVER (PARTITION BY TeamGroup ORDER BY points DESC, Goal_difference DESC, Goals_scored DESC, Matches_won DESC)AS ranks
  FROM group_stage_ranking
)AS top2_teams
WHERE ranks <= 2;

select * from round_of_16;

TeamName,TeamGroup,points,Goal_difference,ranks
France,A,6,-1,1
Belgium,A,5,3,2
Spain,B,5,4,1
Germany,B,5,2,2
Austria,C,9,4,1
Switzerland,C,3,-1,2
Sweden,D,7,4,1
Poland,D,5,4,2
Hungary,E,7,4,1
Slovakia,E,6,3,2


In [0]:
%sql
INSERT INTO round_of_16
SELECT * FROM(
  SELECT TeamName, TeamGroup, points, Goal_difference,
  row_number() OVER (PARTITION BY TeamGroup ORDER BY points DESC, Goal_difference DESC, Goals_scored DESC, Matches_won DESC) AS ranks
  FROM group_stage_ranking
)AS top2_teams
WHERE ranks = 3 LIMIT 4;

select * from round_of_16;

TeamName,TeamGroup,points,Goal_difference,ranks
France,A,6,-1,1
Belgium,A,5,3,2
Spain,B,5,4,1
Germany,B,5,2,2
Austria,C,9,4,1
Switzerland,C,3,-1,2
Sweden,D,7,4,1
Poland,D,5,4,2
Hungary,E,7,4,1
Slovakia,E,6,3,2


In [0]:
%sql
CREATE or replace  TABLE eurocup_third_place_combi(
  ID INTEGER,
  selected_groups VARCHAR(4),
  Team1B CHAR(1),
  Team1C CHAR(1),
  Team1E CHAR(1),
  Team1F CHAR(1)
);

-- Step 2: Insert the provided values into the "THIRDMATCHUP" table
INSERT INTO  eurocup_third_place_combi (ID, selected_groups, Team1B, Team1C, Team1E, Team1F) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');

num_affected_rows,num_inserted_rows
15,15


In [0]:
%sql




In [0]:
%sql
INSERT INTO matches (MatchID, TeamGroup, home_country, away_country, home_country_goal, away_country_goal, winner_country)
WITH combination AS (
  SELECT concat_ws('', sort_array(collect_list(TeamGroup)))AS group_combination
  FROM round_of_16
  WHERE ranks = 3
),
keys AS (
  SELECT * FROM eurocup_third_place_combi
  WHERE selected_groups = (SELECT group_combination FROM combination)
)
SELECT
row_number() OVER (ORDER BY 1) + 36 AS MatchId,
'N' AS Team_group,
r1.TeamName AS home_country,
r2.TeamName AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result
FROM round_of_16 r1
JOIN round_of_16 r2 ON r1.TeamGroup = 'A' AND r1.ranks = 1 AND r2.TeamGroup = 'C' AND r2.ranks = 2


UNION ALL

SELECT 
    row_number() OVER (ORDER BY 1) + 37 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
 round_of_16 r2 ON r1.TeamGroup = 'A' AND r1.ranks = 2 AND r2.TeamGroup = 'B' AND r2.ranks = 2
   

UNION ALL

SELECT 
    row_number() OVER (ORDER BY 1) + 38 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.TeamGroup = 'B' AND r1.ranks = 1 AND r2.TeamGroup = (SELECT TEAM1B FROM keys) AND r2.ranks = 3

UNION ALL

SELECT 
    row_number() OVER (ORDER BY 1) + 39 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.TeamGroup = 'C' AND r1.ranks = 1 AND r2.TeamGroup = (select Team1C from keys) AND r2.ranks = 3

UNION ALL

SELECT 
    Row_Number() OVER (ORDER BY 1) + 40 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
     round_of_16 r2 ON r1.TeamGroup = 'F' AND r1.ranks = 1 AND r2.TeamGroup = (select Team1F from keys) AND r2.ranks = 3


UNION ALL

SELECT 
    Row_Number() OVER (ORDER BY 1) + 41 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.TeamGroup = 'D' AND r1.ranks = 2 AND r2.TeamGroup = 'E' AND r2.ranks = 2
   

UNION ALL

SELECT 
    Row_Number() OVER (ORDER BY 1) + 42 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.TeamGroup = 'E' AND r1.ranks = 1 AND r2.TeamGroup = (SELECT Team1E from keys) AND r2.ranks = 3
   

UNION ALL

SELECT 
    Row_Number() OVER (ORDER BY 1) + 43 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.TeamGroup = 'D' AND r1.ranks = 2 AND r2.TeamGroup = 'F' AND r2.ranks = 2;

num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,4,4,Draw
2,A,France,Belgium,1,1,Draw
3,A,France,Italy,5,4,France
4,A,Italy,Portugal,6,5,Italy
5,A,Italy,Belgium,2,3,Belgium
6,A,Belgium,Portugal,1,5,Portugal
7,B,Spain,England,6,5,Spain
8,B,Spain,Germany,3,5,Germany
9,B,Spain,Netherlands,1,3,Netherlands
10,B,Netherlands,England,3,5,England


In [0]:
%sql
UPDATE matches
SET winner_country = 
CASE
WHEN home_country_goal > away_country_goal THEN home_country
WHEN home_country_goal < away_country_goal THEN away_country
ELSE
CASE
WHEN (home_country_goal + 1) % 2 = 1 THEN home_country
ELSE away_country
END
END

num_affected_rows
44


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,4,4,France
2,A,France,Belgium,1,1,Belgium
3,A,France,Italy,5,4,France
4,A,Italy,Portugal,6,5,Italy
5,A,Italy,Belgium,2,3,Belgium
6,A,Belgium,Portugal,1,5,Portugal
7,B,Spain,England,6,5,Spain
8,B,Spain,Germany,3,5,Germany
9,B,Spain,Netherlands,1,3,Netherlands
10,B,Netherlands,England,3,5,England


Quarter Finals

In [0]:
%sql
INSERT INTO matches (MatchID, TeamGroup, home_country, away_country, home_country_goal, away_country_goal, winner_country)

SELECT
row_number() OVER (ORDER BY 1) + 44 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 37 AND r2.MatchId = 38

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 45 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 39 AND r2.MatchId = 40

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 46 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 41 AND r2.MatchId = 42

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 47 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 43 AND r2.MatchId = 44    


num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,4,4,France
2,A,France,Belgium,1,1,Belgium
3,A,France,Italy,5,4,France
4,A,Italy,Portugal,6,5,Italy
5,A,Italy,Belgium,2,3,Belgium
6,A,Belgium,Portugal,1,5,Portugal
7,B,Spain,England,6,5,Spain
8,B,Spain,Germany,3,5,Germany
9,B,Spain,Netherlands,1,3,Netherlands
10,B,Netherlands,England,3,5,England


In [0]:
%sql
UPDATE matches
SET winner_country = 
    CASE
        WHEN home_country_goal > away_country_goal THEN home_country
        WHEN home_country_goal < away_country_goal THEN away_country
        ELSE 
        CASE 
                WHEN (home_country_goal + 1) % 2 = 1 THEN home_country
                ELSE away_country
            END
    END
     

num_affected_rows
48


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,4,4,France
2,A,France,Belgium,1,1,Belgium
3,A,France,Italy,5,4,France
4,A,Italy,Portugal,6,5,Italy
5,A,Italy,Belgium,2,3,Belgium
6,A,Belgium,Portugal,1,5,Portugal
7,B,Spain,England,6,5,Spain
8,B,Spain,Germany,3,5,Germany
9,B,Spain,Netherlands,1,3,Netherlands
10,B,Netherlands,England,3,5,England


Semi Final

In [0]:
%sql
INSERT INTO matches (MatchID, TeamGroup, home_country, away_country, home_country_goal, away_country_goal, winner_country)

SELECT
row_number() OVER (ORDER BY 1) + 48 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 45 AND r2.MatchId = 46

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 49 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 47 AND r2.MatchId = 48

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
UPDATE matches
SET winner_country = 
    CASE
        WHEN home_country_goal > away_country_goal THEN home_country
        WHEN home_country_goal < away_country_goal THEN away_country
        ELSE 
        CASE 
                WHEN (home_country_goal + 1) % 2 = 1 THEN home_country
                ELSE away_country
            END
    END

num_affected_rows
50


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,4,4,France
2,A,France,Belgium,1,1,Belgium
3,A,France,Italy,5,4,France
4,A,Italy,Portugal,6,5,Italy
5,A,Italy,Belgium,2,3,Belgium
6,A,Belgium,Portugal,1,5,Portugal
7,B,Spain,England,6,5,Spain
8,B,Spain,Germany,3,5,Germany
9,B,Spain,Netherlands,1,3,Netherlands
10,B,Netherlands,England,3,5,England


Third Place

In [0]:
%sql
INSERT INTO matches (MatchID, TeamGroup, home_country, away_country, home_country_goal, away_country_goal, winner_country)
WITH losing_teams AS (
    SELECT
        CASE
            WHEN winner_country != home_country THEN home_country
            ELSE away_country
        END AS losing_teams
    FROM
        matches
    WHERE MatchId IN (49,50)
)

SELECT 
    Row_Number() OVER(ORDER BY 1) + 50 AS match_id,
    'N' as GroupName,
    T1.losing_teams AS home_country,
    T2.losing_teams AS away_country,
    CAST(rand() * 6 AS INTEGER) AS home_country_goal, 
    CAST(rand() * 6 AS INTEGER) AS away_country_goal,
    'None' AS match_results
FROM 
    losing_teams T1
JOIN 
    losing_teams T2 ON T1.losing_teams < T2.losing_teams;

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
UPDATE matches
SET winner_country = 
    CASE
        WHEN home_country_goal > away_country_goal THEN home_country
        WHEN home_country_goal < away_country_goal THEN away_country
        ELSE 
        CASE 
                WHEN (home_country_goal + 1) % 2 = 1 THEN home_country
                ELSE away_country
            END
    END

num_affected_rows
51


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,4,4,France
2,A,France,Belgium,1,1,Belgium
3,A,France,Italy,5,4,France
4,A,Italy,Portugal,6,5,Italy
5,A,Italy,Belgium,2,3,Belgium
6,A,Belgium,Portugal,1,5,Portugal
7,B,Spain,England,6,5,Spain
8,B,Spain,Germany,3,5,Germany
9,B,Spain,Netherlands,1,3,Netherlands
10,B,Netherlands,England,3,5,England


Final

In [0]:
%sql
INSERT INTO matches (MatchID, TeamGroup, home_country, away_country, home_country_goal, away_country_goal, winner_country)

SELECT
row_number() OVER (ORDER BY 1) + 51 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 49 AND r2.MatchId = 50


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
UPDATE matches
SET winner_country = 
    CASE
        WHEN home_country_goal > away_country_goal THEN home_country
        WHEN home_country_goal < away_country_goal THEN away_country
        ELSE 
        CASE 
                WHEN (home_country_goal + 1) % 2 = 1 THEN home_country
                ELSE away_country
            END
    END

num_affected_rows
52


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,4,4,France
2,A,France,Belgium,1,1,Belgium
3,A,France,Italy,5,4,France
4,A,Italy,Portugal,6,5,Italy
5,A,Italy,Belgium,2,3,Belgium
6,A,Belgium,Portugal,1,5,Portugal
7,B,Spain,England,6,5,Spain
8,B,Spain,Germany,3,5,Germany
9,B,Spain,Netherlands,1,3,Netherlands
10,B,Netherlands,England,3,5,England
